Data Injestion

In [2]:
from langchain_core.documents import Document

In [4]:
documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book","pages":1,"author":"Adil Khan"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book","pages":1,"author":"Adil Khan"}),
    Document(page_content="Bananas are yelow", metadata={"title": "banana_book","pages":1,"author":"Adil Khan"}),
]

In [5]:
documents

[Document(metadata={'title': 'apple_book', 'pages': 1, 'author': 'Adil Khan'}, page_content='Apples are red'),
 Document(metadata={'title': 'blueberry_book', 'pages': 1, 'author': 'Adil Khan'}, page_content='Blueberries are blue'),
 Document(metadata={'title': 'banana_book', 'pages': 1, 'author': 'Adil Khan'}, page_content='Bananas are yelow')]

In [6]:
import os
os.makedirs("../data/text_files",exist_ok=True)

In [7]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [10]:
from langchain.document_loaders import TextLoader

loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")

doc=loader.load()

doc

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]